# Reduce number of files of GitHub History, hourly --> daily.

* 24h X 356d x 10y = 87,600 archives
* archive := "JSON Lines (one object per line), compressed.

## 2015-01-01 thru 2024-03-31

In [7]:
!dir z:\gha-raw\2024\2024-02-14*

 Volume in drive Z is 4tb zippy
 Volume Serial Number is E453-D9C7

 Directory of z:\gha-raw\2024

02/13/2024  09:05 PM        93,304,286 2024-02-14-0.json.gz
02/13/2024  10:05 PM        85,665,547 2024-02-14-1.json.gz
02/14/2024  07:05 AM       117,334,207 2024-02-14-10.json.gz
02/14/2024  08:05 AM       112,775,218 2024-02-14-11.json.gz
02/14/2024  09:05 AM       108,883,221 2024-02-14-12.json.gz
02/14/2024  10:05 AM       119,151,659 2024-02-14-13.json.gz
02/14/2024  11:05 AM       125,767,947 2024-02-14-14.json.gz
02/14/2024  12:05 PM       126,953,370 2024-02-14-15.json.gz
02/14/2024  01:05 PM       126,527,862 2024-02-14-16.json.gz
02/14/2024  02:05 PM       119,021,871 2024-02-14-17.json.gz
02/14/2024  03:05 PM       120,025,732 2024-02-14-18.json.gz
02/14/2024  04:05 PM       125,601,768 2024-02-14-19.json.gz
02/13/2024  11:05 PM        70,788,528 2024-02-14-2.json.gz
02/14/2024  05:05 PM       102,158,504 2024-02-14-20.json.gz
02/14/2024  06:05 PM       106,058,584 2024-02-14-

### gharchive-gz-hour2day.py - code listing

* The script is reproduced here for reference (and to leverage JupyterLab + Quarto later :) )
*  This script also works on other platforms w/ linux paths.

```python
import argparse
import os
import glob
from pqdm.processes import pqdm
import logging
from pathlib import Path
from dataclasses import dataclass
from typing import List

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

@dataclass
class ArchiveMetrics:
    """Metrics for daily archive creation."""
    day: str
    missing_files: List[str]
    total_files: int
    total_size: int

@dataclass
class ArchiveArgs:
    """Arguments for daily archive creation."""
    workdir: str
    day: str
    outputdir: str
    dry_run: bool
    verbose: bool
    keep: bool

def create_daily_archive(args: ArchiveArgs) -> ArchiveMetrics:
    fullday_archive = os.path.join(args.outputdir, f"{args.day}.json.gz")
    missing_files = []
    total_files = 0
    total_size = 0
    for hour in range(24):
        hourly_archive = os.path.join(args.workdir, f"{args.day}-{hour}.json.gz")
        if not os.path.exists(hourly_archive):
            missing_files.append(hourly_archive)
            continue
        total_files += 1
        if not args.dry_run:
            # archives are not very large, this approach is simple and fast
            with open(hourly_archive, 'rb') as input:
                with open(fullday_archive, 'ab') as output:
                    data = input.read()
                    output.write(data)
                    total_size += len(data)
                    
    if not missing_files and not args.keep:
        for hour in range(24):
            hourly_archive = os.path.join(args.workdir, f"{args.day}-{hour}.json.gz")
            if os.path.exists(hourly_archive):
                os.remove(hourly_archive)
    return ArchiveMetrics(args.day, missing_files, total_files, total_size)


def generate_summary_report(metrics_list: List[ArchiveMetrics]) -> None:
    """Generate a summary report of the archive creation process."""
    total_days = len(metrics_list)
    total_files = sum(metrics.total_files for metrics in metrics_list)
    total_size = sum(metrics.total_size for metrics in metrics_list)
    print(f"Total Days Processed: {total_days}")
    print(f"Total Files Processed: {total_files}")
    print(f"Total Size Processed: {total_size / (1024 ** 2)} MB")

def process_day(args: ArchiveArgs) -> ArchiveMetrics:
    return create_daily_archive(args)

def discover_days(path_to_hourly_archives: str) -> list:
    days = set()
    for fn in glob.glob(f"{path_to_hourly_archives}/*-0.json.gz"):
        day = os.path.split(fn)[1].removesuffix('-0.json.gz')
        days.add(day)
    return list(days)

def main(workdir: str, outputdir: str, ncores: int, dry_run: bool, verbose: bool, generate_report: bool, keep: bool) -> None:
    if not os.path.exists(outputdir):
        os.makedirs(outputdir)
    days = discover_days(workdir)
    args_list = [ArchiveArgs(workdir, day, outputdir, dry_run, verbose, keep) for day in days]
    metrics_list = pqdm(args_list, process_day, n_jobs=ncores, desc='Processing days')
    if generate_report:
        generate_summary_report(metrics_list)

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Create daily archives from hourly archives.')
    parser.add_argument('workdir', type=str, help='Directory containing hourly archives.')
    parser.add_argument('outputdir', type=str, help='Directory to output daily archives.')
    parser.add_argument('-n', '--ncores', type=int, default=min(os.cpu_count(), 60), help='Number of parallel processes to use.')
    parser.add_argument('-d', '--dry-run', action='store_true', help='Perform a dry run without creating files.')
    parser.add_argument('-v', '--verbose', action='store_true', help='Enable verbose output.')
    parser.add_argument('-r', '--generate-report', action='store_true', help='Generate a summary report after processing.')
    parser.add_argument('-k', '--keep', action='store_true', help='Keep original hourly files after daily archive created.')
    args = parser.parse_args()
    
    main(args.workdir, args.outputdir, args.ncores, args.dry_run, args.verbose, args.generate_report, args.keep)
```

## Results

_Note: this works on macbooks, linux, etc. In this case I'm leveraging more cores and enthusiast harware :)_

```bash
(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>wmic cpu get Name,NumberOfCores,NumberOfLogicalProcessors /format:list

Name=AMD Ryzen Threadripper 3970X 32-Core Processor
NumberOfCores=32
NumberOfLogicalProcessors=64
```

```bash
(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py --help
usage: gharchive-gz-hour2day.py [-h] [-n NCORES] [-d] [-v] [-r] [-k] workdir outputdir

Create daily archives from hourly archives.

positional arguments:
  workdir               Directory containing hourly archives.
  outputdir             Directory to output daily archives.

options:
  -h, --help            show this help message and exit
  -n NCORES, --ncores NCORES
                        Number of parallel processes to use.
  -d, --dry-run         Perform a dry run without creating files.
  -v, --verbose         Enable verbose output.
  -r, --generate-report
                        Generate a summary report after processing.
  -k, --keep            Keep original hourly files after daily archive created.
```

**_To make it simple to reproduce results:_**

```bash
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2015 q:\gha-raw-daily\2015
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2016 q:\gha-raw-daily\2016
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2017 q:\gha-raw-daily\2017
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2018 q:\gha-raw-daily\2018
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2019 q:\gha-raw-daily\2019
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2020 q:\gha-raw-daily\2020
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2021 q:\gha-raw-daily\2021
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2022 q:\gha-raw-daily\2022
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2023 q:\gha-raw-daily\2023
python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2024 q:\gha-raw-daily\2024
```


```bash
(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2015 q:\gha-raw-daily\2015
QUEUEING TASKS | Processing days: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 897.18it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [01:38<00:00,  3.70it/s]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 720094.52it/s]
Total Days Processed: 365
Total Files Processed: 8760
Total Size Processed: 74956.48571968079 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2016 q:\gha-raw-daily\2016
QUEUEING TASKS | Processing days: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 366/366 [00:00<00:00, 908.44it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 366/366 [02:08<00:00,  2.85it/s]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 366/366 [00:00<?, ?it/s]
Total Days Processed: 366
Total Files Processed: 8783
Total Size Processed: 113475.93349266052 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2017 q:\gha-raw-daily\2017
QUEUEING TASKS | Processing days: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 960.13it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [02:44<00:00,  2.22it/s]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 365113.51it/s]
Total Days Processed: 365
Total Files Processed: 8760
Total Size Processed: 146492.90541553497 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2018 q:\gha-raw-daily\2018
QUEUEING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [00:00<00:00, 1005.20it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [03:53<00:00,  1.56it/s]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [00:00<?, ?it/s]
Total Days Processed: 364
Total Files Processed: 8735
Total Size Processed: 178649.27901935577 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2019 q:\gha-raw-daily\2019
QUEUEING TASKS | Processing days: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [00:00<00:00, 960.67it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [04:56<00:00,  1.23it/s]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [00:00<?, ?it/s]
Total Days Processed: 364
Total Files Processed: 8718
Total Size Processed: 257953.86243152618 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2020 q:\gha-raw-daily\2020
QUEUEING TASKS | Processing days: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [00:00<00:00, 994.00it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [08:20<00:00,  1.37s/it]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 364/364 [00:00<?, ?it/s]
Total Days Processed: 364
Total Files Processed: 8710
Total Size Processed: 428740.7561445236 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2021 q:\gha-raw-daily\2021
QUEUEING TASKS | Processing days: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 359/359 [00:00<00:00, 975.03it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 359/359 [09:51<00:00,  1.65s/it]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 359/359 [00:00<?, ?it/s]
Total Days Processed: 359
Total Files Processed: 8528
Total Size Processed: 524822.7800941467 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2022 q:\gha-raw-daily\2022
QUEUEING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 1080.37it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [11:55<00:00,  1.96s/it]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<?, ?it/s]
Total Days Processed: 365
Total Files Processed: 8760
Total Size Processed: 671268.2549409866 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2023 q:\gha-raw-daily\2023
QUEUEING TASKS | Processing days: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<00:00, 997.56it/s]
PROCESSING TASKS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [12:21<00:00,  2.03s/it]
COLLECTING RESULTS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [00:00<?, ?it/s]
Total Days Processed: 365
Total Files Processed: 8760
Total Size Processed: 671390.965722084 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>python gharchive-gz-hour2day.py -k -r -v p:\gha-raw\2024 q:\gha-raw-daily\2024
QUEUEING TASKS | Processing days: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:00<00:00, 295.91it/s]
PROCESSING TASKS | Processing days: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [03:31<00:00,  2.33s/it]
COLLECTING RESULTS | Processing days: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 91/91 [00:00<?, ?it/s]
Total Days Processed: 91
Total Files Processed: 2161
Total Size Processed: 188541.4703245163 MB

(venv-lg) C:\Users\halcy\gh\cncf\landscape-graph\db\scm\sgm-gharchive>
```


In [32]:
!dir q:\gha-raw-daily

 Volume in drive Q is 990 4tb
 Volume Serial Number is C498-1B71

 Directory of q:\gha-raw-daily

04/02/2024  10:21 AM    <DIR>          .
04/02/2024  10:21 AM    <DIR>          ..
04/02/2024  10:21 AM    <DIR>          2015
04/02/2024  10:35 AM    <DIR>          2016
04/02/2024  10:37 AM    <DIR>          2017
04/02/2024  10:40 AM    <DIR>          2018
04/02/2024  10:44 AM    <DIR>          2019
04/02/2024  10:49 AM    <DIR>          2020
04/02/2024  10:57 AM    <DIR>          2021
04/02/2024  11:07 AM    <DIR>          2022
04/02/2024  11:19 AM    <DIR>          2023
04/02/2024  11:48 AM    <DIR>          2024
               0 File(s)              0 bytes
              12 Dir(s)  584,476,262,400 bytes free


In [15]:
!dir q:\gha-raw-daily\2024\2024-02-14*

 Volume in drive Q is 990 4tb
 Volume Serial Number is C498-1B71

 Directory of q:\gha-raw-daily\2024

04/02/2024  11:51 AM     2,440,651,600 2024-02-14.json.gz
               1 File(s)  2,440,651,600 bytes
               0 Dir(s)  584,476,262,400 bytes free


In [28]:
!du -l 1 -nobanner q:\gha-raw-daily\

  76,755,441  q:\gha-raw-daily\2015
 116,199,355  q:\gha-raw-daily\2016
 150,008,735  q:\gha-raw-daily\2017
 182,936,861  q:\gha-raw-daily\2018
 264,144,755  q:\gha-raw-daily\2019
 439,030,534  q:\gha-raw-daily\2020
 537,418,526  q:\gha-raw-daily\2021
 687,378,693  q:\gha-raw-daily\2022
 687,504,348  q:\gha-raw-daily\2023
 193,066,465  q:\gha-raw-daily\2024
Files:        3368
Directories:  11
Size:         3,414,470,367,175 bytes
Size on disk: 3,416,251,170,816 bytes



Processing...

